# Few-shot text classification with OpenAI's GPT models

This notebook illustrates how to use OpenAI's GPT4o model for few-shot text classification.

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI

import re

In [3]:
load_dotenv()
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [15]:
MODEL = 'gpt-4o-2024-08-06' # currently the latest version of GPT-4o (on 2024-09-25)

In [7]:
data_path = os.path.join('..', 'data', 'labeled', 'benoit_crowdsourced_2016', '')

In [10]:
SEED=42

## Define the task

In this example, we adapt the instruction for the economic/social/neither policy area classification task in Benoit et al. (2016)

- see [this README file](../data/labeled/benoit_crowdsourced_2016/README.md) for a description of the data and tasks covered in the paper
- see [this file](../data/labeled/benoit_crowdsourced_2016/instructions/econ_social_policy.md) for a copy of their original task instructions

In [5]:
instructions = """
This task involves reading sentences from political texts and judging whether these deal with economic or social policy.

The sentences you will be asked to interpret come from political party manifestos.
Some of these sentences will deal with economic policy; some will deal with social policy; other sentences will deal with neither economic nor social policy. We tell you below about what we mean by "economic" and "social" policy.

First, you will read a short section from a party manifesto.
For the sentence highlighted in red, enter your best judgment about whether it mainly refers to economic policy, to social policy, or to neither.

If the sentence refers to economic policy, select "economic" in the drop down menu; if it refers to social policy, select "social".
If the sentence does not refer to either policy area, select "Not Economic or Social" -- in this case you will move directly to the next sentence.

Now we need to tell you about what we mean by "economic" and "social" policy.

## What is "economic" policy?

**"Economic" policies** deal with all aspects of the economy, including:

- Taxation
- Government spending
- Services provided by the government or other public bodies
- Pensions, unemployment and welfare benefits, and other state benefits
- Property, investment and share ownership, public or private
- Interest rates and exchange rates
- Regulation of economic activity, public or private
- Relations between employers, workers and trade unions

## What is "social" policy?

**"Social" policies** deal with aspects of social and moral life, relationships between social groups, and matters of national and social identity, including:

- Policing, crime, punishment and rehabilitation of offenders;
- Immigration, relations between social groups, discrimination and multiculturalism;
- The role of the state in regulating the social and moral behavior of individuals

## Your task

Classify the input text in one of the following categories: economic, social, neither

## Response format

Only respond with the chosen category and no additional text or explanations 
"""

### Load the data

In [23]:
from utils.io import read_tabular
from utils.finetuning import split_data

fp = data_path+'benoit_crowdsourced_2016-policy_area.csv'
df = read_tabular(fp, columns=['text', 'label'])

id2label = {
    2: "economic",
    3: "social",
    1: "neither"
}

df.label = df.label.map(id2label)

# subset to 100 examples per label class 
df = df.groupby('label').sample(n=100, random_state=SEED)

In [24]:
df.label.value_counts()

label
economic    100
neither     100
social      100
Name: count, dtype: int64

In [26]:
# split into "training" and "test" data
#  - training is used for examples
#  - test is used for few-shot classification  
# note: if you had different example selection 
#  strategies of variations of a prompt, you could use a dev for comparing them
data_splits = split_data(df, test_size=0.5, dev_size=None, stratify_by='label', return_dict=True)
data_splits.keys()

dict_keys(['train', 'test'])

## sample examples

There is various different ways of sampling few-shot exemplars

Here, we take the **most representative exemplars** by 

1. embedding exemplars
2. computing the centroid for each label class
3. and ranking exemplars in terms of their closeness to their class centroid

In [27]:
import numpy as np
from typing import List
def embed(texts: List[str]):
    # see https://platform.openai.com/docs/guides/embeddings/what-are-embeddings?lang=python
    texts = [text.replace("\n", " ") for text in texts] 
    res = client.embeddings.create(
        input=texts,
        model='text-embedding-3-small'
    )
    embeddings = [(e.index, e.embedding) for e in res.data]
    # sort embeddings by index
    embeddings = [e[1] for e in sorted(embeddings, key=lambda x: x[0])]
    return np.array(embeddings)

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

ranked_exemplars = {}
for label_class, df in data_splits['train'].groupby('label'):
    # get embeddings from OpenAI model
    embeddings = embed(df.text.to_list())
    # compute centroid
    centroid = embeddings.mean(axis=0)
    # compute cosine similarity between the centroid and the embeddings
    dists = cosine_similarity([centroid], embeddings)
    # rank the examples by similarity, descending
    ranked_indices = np.argsort(dists[0])[::-1]
    # add to output
    ranked_exemplars[label_class]= df.iloc[ranked_indices, 0]

In [50]:
import random
def get_n_examples(ranked_exemplars, k, shuffle=True, random_state=SEED):
    exs = []
    for label_class, exemplars in ranked_exemplars.items():
        ex = exemplars[:k].to_list()
        ex = [{'text': text, 'label': label_class} for text in ex]
        exs.extend(ex)
    if shuffle:
        random.Random(random_state).shuffle(exs)
    return exs

In [51]:
get_n_examples(ranked_exemplars, 3)

[{'text': 'Liberal Democrats are determined that Britain should lead this reform.',
  'label': 'neither'},
 {'text': 'We believe in being tough on crime and its causes so we will expand drugs testing and treatment, and tackle the conditions -from lack of youth provision to irresponsible drinking- that foster crime and anti-social behaviour.',
  'label': 'social'},
 {'text': 'We will audit the resources available, take proper ministerial responsibility for the service, and seek to ensure that prison regimes are constructive and require inmates to face up to their offending behaviour.',
  'label': 'social'},
 {'text': "It's a new chance to win friends abroad. Labour will end Tory government meddling in the valuable work of the British Council.",
  'label': 'neither'},
 {'text': 'We will consult on introducing indefinite sentences for the most serious sex and violent offenders, so that they would only be released following an assessment by the court of the risk which they pose to society.

In [72]:
def convert_exemplars_list_to_convo(exemplars):
    convo = []
    for ex in exemplars:
        convo.append({"role": "user", "content": f"'''{ex['text']}'''"},)
        convo.append({"role": "assistant", "content": ex['label']},)
    return convo

In [73]:
convert_exemplars_list_to_convo(get_n_examples(ranked_exemplars, 3))[:6]

[{'role': 'user',
  'content': "'''Liberal Democrats are determined that Britain should lead this reform.'''"},
 {'role': 'assistant', 'content': 'neither'},
 {'role': 'user',
  'content': "'''We believe in being tough on crime and its causes so we will expand drugs testing and treatment, and tackle the conditions -from lack of youth provision to irresponsible drinking- that foster crime and anti-social behaviour.'''"},
 {'role': 'assistant', 'content': 'social'},
 {'role': 'user',
  'content': "'''We will audit the resources available, take proper ministerial responsibility for the service, and seek to ensure that prison regimes are constructive and require inmates to face up to their offending behaviour.'''"},
 {'role': 'assistant', 'content': 'social'}]

### A single text example

In [74]:
text_input = data_splits['test'].text.values[0]

In [75]:
# convert to conversation history
messages = [
  # system prompt
  {"role": "system", "content": instructions},
  # exemplars
  *convert_exemplars_list_to_convo(get_n_examples(ranked_exemplars, 3))[:6],
  # user input
  {"role": "user", "content": f"'''{text_input}'''"},
]

In [76]:
response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  # for reproducibility
  temperature=0.0,
  seed=42,
)

In [77]:
# parse the response
response.choices[0].message.content

'neither'

In [78]:
data_splits['test'].label.values[0]

'neither'

### Iterate over multiple examples

Let's first define a custom function to classify tweets:

In [79]:
from typing import List, Dict
def classify_tweet(text, model: str, system_message: str, exemplars: List[Dict]):

  # clean the text 
  text = re.sub(r'\s+', ' ', text).strip()

  # construct input

  messages = [
    # system prompt
    {"role": "system", "content": system_message},
    # exemplars
    *convert_exemplars_list_to_convo(exemplars),
    # user input
    {"role": "user", "content": f"'''{text}'''"},
  ]

  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.0,
    seed=42
  )
  
  if len(response.choices) != 1:
      print("WARNING: Response should have one 'choice'")
      return None
  if response.choices[0].finish_reason != 'stop':
      print("WARNING: Response should have 'finish_reason' of 'stop' but got:", response.choices[0].finish_reason)
      return None

  result = response.choices[0].message.content
  
  return result

Now we can iterate over example texts:

In [83]:
from tqdm.notebook import tqdm

# with GPT 4o turbo
texts = data_splits['test'].text.to_list()
exemplars = get_n_examples(ranked_exemplars, 3)
classifications = [classify_tweet(text, model=MODEL, system_message=instructions, exemplars=exemplars) for text in tqdm(texts)]
classifications

  0%|          | 0/150 [00:00<?, ?it/s]

['neither',
 'economic',
 'social',
 'economic',
 'social',
 'neither',
 'economic',
 'economic',
 'social',
 'social',
 'social',
 'social',
 'neither',
 'neither',
 'neither',
 'social',
 'neither',
 'economic',
 'neither',
 'economic',
 'neither',
 'social',
 'social',
 'social',
 'neither',
 'economic',
 'economic',
 'neither',
 'economic',
 'social',
 'neither',
 'economic',
 'social',
 'neither',
 'social',
 'social',
 'social',
 'social',
 'social',
 'neither',
 'social',
 'social',
 'economic',
 'social',
 'neither',
 'economic',
 'economic',
 'economic',
 'neither',
 'social',
 'economic',
 'neither',
 'economic',
 'neither',
 'neither',
 'neither',
 'economic',
 'neither',
 'neither',
 'economic',
 'economic',
 'economic',
 'neither',
 'neither',
 'economic',
 'economic',
 'economic',
 'social',
 'neither',
 'social',
 'neither',
 'economic',
 'economic',
 'neither',
 'social',
 'economic',
 'neither',
 'social',
 'economic',
 'social',
 'neither',
 'economic',
 'economic',
 

In [84]:
from sklearn.metrics import classification_report

cr = classification_report(
    y_true = data_splits['test'].label.to_list(),
    y_pred = classifications
)

print(cr)

              precision    recall  f1-score   support

    economic       0.81      0.88      0.85        50
     neither       0.76      0.84      0.80        50
      social       0.95      0.78      0.86        50

    accuracy                           0.83       150
   macro avg       0.84      0.83      0.83       150
weighted avg       0.84      0.83      0.83       150

